# This Notebook demos retreival of training data from AWS feature store into CAS in-memory.
### Type 1. Retreive data from feature store into a panda dataframe, save data into .csv file and then load csv file. 
### Type 2. Explore featuregroup settings to find underlying S3 data location and then load data directly.
### Type 3. Use Hive/EP if you have hive cluster running. This probably fastest way to load data from feature store 
### Type 4. This is for online feature store record access. Ideal for fast data access during inference. It reads only the latest row(s) identified by record_identifier.
   

### Step 0 - This sets default properties for remainder of notebook

In [ ]:
# default_bucket for everything.
default_bucket = "fsbu-user1-s3bucket-1"

# offline store area for feature-store. - You do not need this ..
## feature_store_s3_bucket = "fsbu-user1-feature-store"

# feature group we are reading and loading into CAS in-memory table. 
feature_group_name = "fraud-summary-feature-group"

# All access defined on this role
sagemaker_iam_role='arn:aws:iam::123456789012:role/fsbu-user1-sagemaker-fullaccess'

# following are for SAS/SWAT
cashost = "fsds-viya34lab-controller.fsl.sashq-d.openstack.sas.com"
#cashost = "fsbuviya4.fsbu-openstack-k8s.unx.sas.com"
casport = 5570
protocol = "cas"
username = "user1"

# python boto3 picks creds from $home/.aws/config and credentials. 
# SWAT Viya4 operates on K8S and cannot see home directories. so we need folloowing for Viya4/K8S aws access. 
# CAS ctontroller pods should see these PVC - you know what I am talking about.
aws_config = "/datasourcelib/user1/aws/config"
aws_credentials = "/datasourcelib/user1/aws/credentials"
aws_config = "/sgrid/home/user1/.aws/config"
aws_credentials = "/sgrid/home/user1/.aws/credentials"
#########


### Step 1 - The following cell is common for all types. Basically creates a CAS connection and boto3 connection


In [69]:
# This cell is common and required for all 3 types of data access.

import numpy as np
import pandas as pd
import boto3

import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

boto3_session = boto3.Session()
s3_client = boto3_session.client('s3')
sagemaker_client = boto3_session.client('sagemaker')
featurestore_runtime = boto3_session.client(service_name="sagemaker-featurestore-runtime")

feature_store_session = Session(boto_session=boto3_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

fraud_summary_feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=feature_store_session)

import os
import time
import swat
import pandas
import json
from getpass import getpass

#os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/sgrid/openssl_certs/cas_controller_certs/fsds-viya34lab-controller.pem'
#os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/sgrid/openssl_certs/cas_controller_certs/fsbuviya4.fsbu-openstack-k8s.unx.sas.com.pem'
#os.environ['TKESSL_OPENSSL_LIB'] = '/usr/lib64/libssl.so.10'
os.environ['SSLREQCERT']='ALLOW'  
## ==> if you want to ignore cert business.

password = getpass()
#password = ""
conn = swat.CAS(cashost, casport, protocol=protocol,username=username,password=password)
out = conn.serverstatus()
print("status : ", out)



 ···········


NOTE: Grid node action status report: 5 nodes, 8 total actions executed.
status :  [About]

 {'CAS': 'Cloud Analytic Services',
  'Version': '3.05',
  'VersionLong': 'V.03.05M0P11112019',
  'Copyright': 'Copyright © 2014-2018 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2021-08-12T02:12:27Z',
  'System': {'Hostname': 'fsds-viya34lab-controller.fsl.sashq-d.openstack.sas.com',
   'OS Name': 'Linux',
   'OS Family': 'LIN X64',
   'OS Release': '3.10.0-1062.1.1.el7.x86_64',
   'OS Version': '#1 SMP Fri Sep 13 22:55:44 UTC 2019',
   'Model Number': 'x86_64',
   'Linux Distribution': 'CentOS Linux release 7.7.1908 (Core)'},
  'license': {'site': 'Viya 3.5 FSBU GA Shipped',
   'siteNum': 70180938,
   'expires': '11Feb2022:00:00:00',
   'gracePeriod': 45,
   'warningPeriod': 45}}

[server]

 Server Status
 
    nodes  actions
 0      5        8

[nodestatus]

 Node Status
 
                                                 name        role  uptime  \
 0  fsds-viya34lab-worker-1.fs

## Type 1 to load data into Viya in-memory table. 
##### Retrieve data into panda data frame, keep columns you want and then save into a S3 file. Then load that S3 file into CAS-in-memory
##### Time it takes to read data. 

In [70]:
# read data into a panda data frame and then into a CSV and finally into CAS table. 
# Time the whole operation which means run this cell in it's entirety 

start_time = time.time()

a_query = fraud_summary_feature_group.athena_query()
a_table = a_query.table_name
a_query_string = ('select * from "' + a_table + '"' )
a_query.run(query_string=a_query_string, output_location= "s3://" + default_bucket + "/query_results")
a_query.wait()
a_pd = pd.DataFrame()
a_pd = a_query.as_dataframe()
print(dir(a_query))
print("pd frame shape :", a_pd.shape)

# save data into a CSV file
a_pd = a_pd [
    [
        "auth_account_id",
        "auth_date",
        "daily_sum",
        "daily_count"
    ]
]
a_pd.to_csv("../data/temp_fraud_summary.csv", header=False,index=False)

# read CSV file into CAS memory 
tbl = conn.read_csv('../data/temp_fraud_summary.csv',casout={"replication":'0'})
#print(conn.table.fileinfo(caslib='CASUSERHDFS'))
print(conn.table.tableinfo(caslib='CASUSERHDFS'))

# print the time 
print("Type 1 - Total time for reading feature store data --- %s seconds ---" % (time.time() - start_time))


['__annotations__', '__attrs_attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_current_query_execution_id', '_result_bucket', '_result_file_prefix', 'as_dataframe', 'catalog', 'database', 'get_query_execution', 'run', 'sagemaker_session', 'table_name', 'wait']
pd frame shape : (171428, 8)
NOTE: Cloud Analytic Services made the uploaded file available as table TMPJUWJ8KLW in caslib CASUSERHDFS(user1).
NOTE: The table TMPJUWJ8KLW has been created in caslib CASUSERHDFS(user1) from binary data uploaded to Cloud Analytic Services.
[TableInfo]

           Name    Rows  Columns  IndexedColumns Encoding  \
 0  TMPJUWJ8KLW  171427        4               0    utf-8   
 
          CreateTimeForma

## Type 2 to load data into Viya in-memory table. 
##### Retrieve feature data right of offline feature store underlying S3 folder. 
##### All data is stored as parquets.. so use recurse = true during loadTable action into CAS-in-memory
##### Time it takes to read data. 
##### We need to able to recursively read partitioned parquet datasets.
##### Viya 2021.1.3 does not support and so this needs to wait until that functionality is same as that of CSV files(see earlier cell)

In [71]:
## TEST CELL - do not use it . This is just for testing multi-file load on CSV's 
### DO NOT RUN. IGNORE ERRORS IF ANY
#NO NEED TO RUN THIS. JUST A TEST SETUP FOR CSVS

# split bucket and relative folder 
feature_group_bucket = "fsbu-user1-s3bucket-1"
feature_group_s3_prefix = "s3-test-folder"

print("bucket: ", feature_group_bucket)
print("key: ", feature_group_s3_prefix )

datasource = "{\"srctype\":\"s3\",\"bucket\":\"" + feature_group_bucket + "\", \"objectPath\":\"/" + feature_group_s3_prefix + "/\", \"awsConfigPath\": '" + aws_config + "', \"awsCredentialsPath\": '" + aws_credentials + "' } " 
print ("ds: ", datasource)

# make sure subdirectories is true.
conn.table.addCaslib(name='S3CASLIBx', description='', subDirectories='True', session='true', activeOnAdd='true', dataSource=eval(datasource), createDirectory='false')
conn.table.loadtable(caslib='s3CASLIBx',path="sub-folder-1",importOptions={"fileType":"csv","multiFile":"True","recurse":"True"})
#conn.table.dropcaslib(caslib='s3caslib3')


bucket:  fsbu-user1-s3bucket-1
key:  s3-test-folder
ds:  {"srctype":"s3","bucket":"fsbu-user1-s3bucket-1", "objectPath":"/s3-test-folder/", "awsConfigPath": '/sgrid/home/user1/.aws/config', "awsCredentialsPath": '/sgrid/home/user1/.aws/credentials' } 
NOTE: 'S3CASLIBx' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 'S3CASLIBx'.
NOTE: The file, '/s3-test-folder/sub-folder-1/sub-folder-2/test_1b.csv' was used to create the CAS Table column names.
NOTE: The CSV file table load for table, 'sub-folder-1' produced 60 rows from 3 files.
NOTE: Cloud Analytic Services made the file sub-folder-1 in AWS S3 bucket fsbu-user1-s3bucket-1 available as table SUB-FOLDER-1 in caslib S3CASLIBx.


[caslib]

 'S3CASLIBx'

[tableName]

 'SUB-FOLDER-1'

[casTable]

 CASTable('SUB-FOLDER-1', caslib='S3CASLIBx')

+ Elapsed: 0.777s, user: 0.156s, sys: 0.139s, mem: 264mb

In [72]:
# Use boto3 and sagemaker sdk to get feature group settings of where offline data is saved
# Read the data straight out of S3 using S3 parquet connector.
# SAS Viya version (Viya4 and above) should support recursive partitioned parquet datasets.
# Viya 2021.1.3 does not support and so this needs to wait until that functionality is same as that of CSV files(see earlier cell)
##
##
# Time the whole operation which means run this cell in it's entirety 

start_time = time.time()

fraud_summary_feature_group_resolved_output_s3_uri = (
    fraud_summary_feature_group.describe()
    .get("OfflineStoreConfig")
    .get("S3StorageConfig")
    .get("ResolvedOutputS3Uri")
)
print("S3 path for feature group: ", fraud_summary_feature_group_resolved_output_s3_uri)

# split bucket and relative folder 
feature_group_bucket = fraud_summary_feature_group_resolved_output_s3_uri.split("/")[2]
feature_group_s3_prefix = fraud_summary_feature_group_resolved_output_s3_uri.replace(
    f"s3://{feature_group_bucket}/", ""
)

# following hack to replace "data" at the end since SAS Viya cannot process tag path="." for current directory. it needs a non-null value
# so we replace data with blank only to access it back as path="data" during load table action - sudhir reddy
feature_group_s3_prefix = feature_group_s3_prefix.replace(f"/data", "")

print("bucket: ", feature_group_bucket)
print("key: ", feature_group_s3_prefix )

datasource = "{\"srctype\":\"s3\",\"bucket\":\"" + feature_group_bucket + "\", \"objectPath\":\"/" + feature_group_s3_prefix + "/\", \"awsConfigPath\": '" + aws_config + "', \"awsCredentialsPath\": '" + aws_credentials + "' } " 
#datasource = "{\"srctype\":\"s3\",\"bucket\":\"" + feature_group_bucket + "\", \"objectPath\":\"/" + feature_group_s3_prefix + "/\", \"awsConfigPath\":'/sgrid/home/user1/.aws/config', \"awsCredentialsPath\":'/sgrid/home/user1/.aws/credentials'} " 
print ("ds: ", datasource)

# make sure subdirectories is true.
#conn.table.addCaslib(name='S3CASLIB2', description='', subDirectories='True', session='true', activeOnAdd='true', dataSource=eval(datasource), createDirectory='false')
#conn.table.loadtable(caslib='s3caslib2',path="data",importOptions={"fileType":"PARQUET","multiFile":"True","recurse":"True"},casout={"replication":'0'})
#conn.table.loadtable(caslib='s3caslib2',path="data",importOptions={"fileType":"PARQUET"})
#print(conn.table.tableinfo(caslib='CASUSERHDFS'))

# print the time 
print("Type 2 - Total time for reading feature store data --- %s seconds ---" % (time.time() - start_time))


S3 path for feature group:  s3://fsbu-user1-feature-store/123456789012/sagemaker/us-east-1/offline-store/fraud-summary-feature-group-1628479259/data
bucket:  fsbu-user1-feature-store
key:  123456789012/sagemaker/us-east-1/offline-store/fraud-summary-feature-group-1628479259
ds:  {"srctype":"s3","bucket":"fsbu-user1-feature-store", "objectPath":"/123456789012/sagemaker/us-east-1/offline-store/fraud-summary-feature-group-1628479259/", "awsConfigPath": '/sgrid/home/user1/.aws/config', "awsCredentialsPath": '/sgrid/home/user1/.aws/credentials' } 
Type 2 - Total time for reading feature store data --- 0.20631814002990723 seconds ---


In [73]:
print(conn.table.caslibinfo(caslib='s3caslib2',verbose="True"))
#conn.table.dropcaslib(caslib='s3caslib2')

+ Elapsed: 0.00927s, user: 0.00277s, sys: 0.00948s, mem: 1.42mb


ERROR: The caslib 's3caslib2' does not exist in this session.
ERROR: The action stopped due to errors.


## Type 3 to load data into Viya in-memory table by leveraging EMR. 
##### feature stores use S3 for storage and during creation of feature groups (notebook1) weeeeget option to save that DDL
##### DDL refers to S3 folder as an external table. So We will simply define that table externally outside this note book.We need to define table only one time and not for every updates/loading of feature data .
##### Once We define that table we can load data using SAS Access to hadoop serial as well as parallel to load data.
##### This is probably faster than first two because we can use EMR and EP parallel loading . 

In [74]:

start_time = time.time()

hadoop_schema = "sagemaker_featurestore"
hadoop_table_name = "fraud_summary_feature_group"
hadoop_uid = "hadoop"
data_transfeR_mode = "serial"
hadoop_config_dir = "/sgrid/hadoop_jars_conf/emr-5.33.0/sitexmls"
hadoop_jar_path = "/sgrid/hadoop_jars_conf/emr-old/jars"
# established ssh tunnel from CAS controller to EMR Hive server else latter will not let you make it from on-prem.
hadoop_server = "localhost"

#dataSource={dataTransferMode='Serial', uid='hadoop', hadoopJarPath='/sgrid/hadoop_jars_conf/emr-old/jars', schema='sagemaker_featurestore', properties='hive.txn.manager=org.apache.hadoop.hive.ql.lockmgr.DbTxnManager;hive.support.concurrency=true', allFSTypes=true, srcType='hadoop', hadoopConfigDir='/sgrid/hadoop_jars_conf/emr-5.33.0/sitexmls', server='localhost'}

datasource = "{\"srctype\":\"hadoop\",\"schema\":\"" + hadoop_schema + "\", \"uid\":\"" + hadoop_uid + "\", \"hadoopConfigDir\":'" + hadoop_config_dir + "',\"hadoopJarPath\":'" + hadoop_jar_path + "',\"server\":\'" + hadoop_server + "\'} " 
print ("ds: ", datasource)

conn.table.addCaslib(name='s3caslib3', description='', subDirectories='True', session='true', activeOnAdd='true', dataSource=eval(datasource), createDirectory='false')
conn.table.loadtable(caslib='s3caslib3',path=hadoop_table_name,casout={"replication":'0'})
print(conn.table.tableinfo(caslib='s3caslib3'))

# print the time 
print("Type 2 - Total time for reading feature store data --- %s seconds ---" % (time.time() - start_time))


ds:  {"srctype":"hadoop","schema":"sagemaker_featurestore", "uid":"hadoop", "hadoopConfigDir":'/sgrid/hadoop_jars_conf/emr-5.33.0/sitexmls',"hadoopJarPath":'/sgrid/hadoop_jars_conf/emr-old/jars',"server":'localhost'} 
NOTE: 's3caslib3' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 's3caslib3'.
NOTE: Performing serial LoadTable action using SAS Data Connector to Hadoop.
NOTE: Cloud Analytic Services made the external data from fraud_summary_feature_group available as table FRAUD_SUMMARY_FEATURE_GROUP in caslib s3caslib3.
[TableInfo]

                           Name    Rows  Columns  IndexedColumns Encoding  \
 0  FRAUD_SUMMARY_FEATURE_GROUP  171428        8               0    utf-8   
 
          CreateTimeFormatted           ModTimeFormatted  \
 0  2021-08-11T22:19:59-04:00  2021-08-11T22:19:59-04:00   
 
          AccessTimeFormatted JavaCharSet    CreateTime  ...  Repeated  View  \
 0  2021-08-11T22:19:59-04:00        UTF8  1.944354e+09  ...         0     0

In [75]:
print(conn.table.caslibinfo(caslib='s3caslib3',verbose="True"))
#conn.table.dropcaslib(caslib='s3caslib3')

[CASLibInfo]

         Name    Type Description Path  \
 0  s3caslib3  hadoop                    
 1  s3caslib3                            
 2  s3caslib3                            
 3  s3caslib3                            
 4  s3caslib3                            
 
                                           Definition  Subdirs  Local  Active  \
 0                                     uid = 'hadoop'      1.0    1.0     1.0   
 1  hadoopJarPath = '/sgrid/hadoop_jars_conf/emr-o...      NaN    NaN     NaN   
 2                  schema = 'sagemaker_featurestore'      NaN    NaN     NaN   
 3  hadoopConfigDir = '/sgrid/hadoop_jars_conf/emr...      NaN    NaN     NaN   
 4                               server = 'localhost'      NaN    NaN     NaN   
 
    Personal  Hidden  Transient  
 0       0.0     0.0        0.0  
 1       NaN     NaN        NaN  
 2       NaN     NaN        NaN  
 3       NaN     NaN        NaN  
 4       NaN     NaN        NaN  

+ Elapsed: 0.00953s, user: 0.00382s, sy

In [67]:

#out = conn.serverstatus()
#print("status : ", out)

conn.close()